# Import Libraries

In [1]:
####### INIT ######################################################################################

from context import rmlib
from time import sleep

import numpy as np
import rmlib

from rmlib.rmtools.asm_BT_lib.asm_tree_Basic import augment_RM
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot(robotConfig)
augment_RM(robot)

#%store -r  #loads variables attached to this notebook

####### LIBS ######################################################################################

# Import Pytrees
import py_trees
#from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector

from rmlib.rmtools.asm_BT_lib.asm_tree_Basic import *
from rmlib.rmtools.asm_BT_lib.asm_tree_FT_based import *

import math
from pmath import translate_pose, get_disance_between_poses, pose_components

from utils import is_matx_list
from math import radians

print( "\n##!## READY ##!##" )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Setting up robot please wait...
my_components: {'hand_config': {'finger_width_outer': 0.015, 'finger_length': 0.0415, 'module_name': 'hands.smarthand', 'finger_depth': 0.014, 'class_name': 'SmartHand', 'finger_width_inner': 0.0}, 'arm_config': {'xmlrpc_port': '8003', 'max_linear_accel': 1.2, 'module_name': 'arms.ur5', 'ip_address': '192.168.0.101', 'class_name': 'UR5', 'max_joint_speed': 1.05, 'default_linear_accel': 0.8, 'max_linear_speed': 0.25, 'default_joint_accel': 0.8, 'max_joint_accel': 1.4, 'default_joint_speed': 0.7, 'default_linear_speed': 0.1}, 'ft_config': {'local_port': 20000, 'max_workers': 10, 'module_name': 'sensors.ftsensor_optoforce', 'ip_address': '192.168.0.100', 'local_ip': '127.0.0.1', 'class_name': 'OptoForce', 'remote_ip': '', 'dataHz': 50.0, 'filter': {'alpha': 0.012, 'beta': 0.035}, 'remote_port': 10000}}
active_componetns: {'hand': ['hand_

In [2]:

SafePose=np.array([[-0.99934454 ,-0.00727455 , 0.03546237 ,-0.13835163],
 [-0.00685909 , 0.99990658 , 0.01182325 ,-0.30214523],
 [-0.03554506 , 0.01157226 ,-0.99930107 , 0.24818522],
 [ 0.         , 0.      ,    0.      ,    1.        ]])

BlockInsertPose=np.array([[-0.99937302 ,-0.00788234 , 0.03451718 ,-0.23715425],
 [-0.00746342,  0.99989713 , 0.01224868 ,-0.60009125],
 [-0.03461017 , 0.01198338, -0.99932904 , 0.06395784],
 [ 0.         , 0.         , 0.          ,1.        ]])

In [3]:
print(SafePose)

[[-0.99934454 -0.00727455  0.03546237 -0.13835163]
 [-0.00685909  0.99990658  0.01182325 -0.30214523]
 [-0.03554506  0.01157226 -0.99930107  0.24818522]
 [ 0.          0.          0.          1.        ]]


In [4]:
#from task_parameters import taskParams , _TB_side , _M8_TB_height , _M6_TB_height , _M4_TB_height

_DRILL_CONTACT_PRESS = 11.0

#SafePose = robot.arm.get_tcp_pose() #rm.get_tcp_pose()
#%store SafePose

#BlockInsertPose = robot.arm.get_tcp_pose() #rm.get_tcp_pose()
#%store BlockInsertPose
#no stored variable or alias this

#%store -r 

"""
### Move to Safe Pose ###
Basic movements are defined in:
rmlib/rmlib/rmtools/assembly_trees/asm_tree_Basic.py

All of the basic behaviors and trees inherit `py_trees` classes.

"""
moveNode = Move_Arm( pose  = SafePose , # 4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.125 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref togitkraken `rmlib` object, stored in `robot` in this NB
run_BT_until_done(moveNode,HidePrintout=1)#Nverb=0,
 
print('done!')

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
moveNode = Move_Arm( pose  = BlockInsertPose, # BlockInsertPose  SafePose  4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.125 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
run_BT_until_done(moveNode,HidePrintout=1)
print("Done!")


# Arm Movements

In [ ]:
def exceeds_Z_force2(zLimitF):
        #Condition: Return a function that returns 1 if the z reaction limit is reached 
        def func():
            _DEBUG = 0
            #Stops when Z reaction exceeds value
            wrench = robot.ft.get_wrist_force()
            mag_x   = wrench[0]
            mag_y   = wrench[1]
            mag_z   = wrench[2]
            if _DEBUG: 
                print( "exceeds_Z_force , wrench:" , wrench )
                print("wrench[2]=",wrench[2], "    -zLimitF=",-zLimitF)
            if wrench[2] < -zLimitF:
                builtins._GLOB_FT_FLAG = 1
                #print("pass")
                return 1
            else:
                #print("fail")
                return 0
        return func
    
def exceeds_force(force):
        #Condition: Return a function that returns 1 if the z reaction limit is reached 
        def func():
            _DEBUG = 0
            #Stops when Z reaction exceeds value
            wrench = robot.ft.get_wrist_force()
            mag_x   = wrench[0]
            mag_y   = wrench[1]
            mag_z   = wrench[2]

            if (abs(wrench[0]) > force or 
                abs(wrench[1]) > force or 
                abs(wrench[2]) > force) :
                builtins._GLOB_FT_FLAG = 1
                #print("pass")
                return 1
            else:
                #print("fail")
                if _DEBUG: 
                    print( "exceeds force limit of ", force ,"N  , wrench:" , wrench )
                return 0
        return func     
    

## Command jog + angle loop, maxed at 2N for any move

In [ ]:

import socket
import numpy as np
import struct


relMoveDistance=0.0254/16  # 1 inch in meters
rotation_angle=5 # in degrees
rotation_angle=rotation_angle*(math.pi/180)  #convert to radians
#maxForce=2.0 #TapFz uses positive values #Newtons
inputstring=0
forcelimit=4.0
#condition = EXCEEDS_Z_force
#print(dir(condition))
robot.ft.bias_wrist_force()   
verbose=0


In [ ]:
from selectors_multi_server import SelectorsMultiServer

import multiprocessing as mp


from selectors_multi_client import SelectorsMultiClient, get_message


name_1_server_address = ("localhost", 9001)
name_2_server_address = ("localhost", 9002)


def name_1_read_callback(key, client, name):
    connection = key.fileobj
    data = connection.recv(1024)
    if data:
        print("  received {!r}".format(data))


def name_1_write_callback(key, client, name):
    connection = key.fileobj
    msg = get_message(client.input_q, timeout=2)
    if msg is not None:
        print(msg)
        connection.sendall(b"START")


def name_2_read_callback(key, client, name):
    connection = key.fileobj
    data = connection.recv(1024)
    cross_message = b"Message initated by name_1, sent to name_2"
    if data:
        print("  received {!r}".format(data))
        if data == cross_message:
            connection.sendall(b"STOP")
            client.running = False


def name_2_write_callback(key, client, name):
    pass


client_data = {
    "name_1": (name_1_read_callback, name_1_write_callback, name_1_server_address),
    "name_2": (name_2_read_callback, name_2_write_callback, name_2_server_address),
}


def main():
    print("testing SelectorsMultiClient with multiprocessing")
    # TODO: write to the input queue using a different process
    # TODO: message acknowledgements and message type scheme
    # for sockets and mp.Queues

    input_q = mp.Queue()
    input_q.put("MSG: start")
    output_q = mp.Queue()

    selectors_args = {
        "input_q": input_q,
        "output_q": output_q,
        "client_data": client_data,
    }

    with SelectorsMultiClient(**selectors_args) as client:
        client.run()

main()


# Utils & Setup

In [ ]:
robot.hand.deactivate_motors()

In [ ]:
robot.arm.get_tcp_pose()

In [ ]:
robot.hand.set_finger_torque(0.3)

In [ ]:
robot.hand.set_finger_width(0.0)

In [ ]:
robot.arm.move_speed( midDrillPose , 'l' , 0.125 , 0.35 , 0 , 'dummy' , False )

In [ ]:
SafePose = robot.arm.get_tcp_pose() #rm.get_tcp_pose()
%store SafePose

In [ ]:
robot.arm.move_speed( aboveTableDrill , 'l' , 0.125 , 0.35 , 0 , 'dummy' , False )

# Run this after teaching any poses

In [ ]:
import pickle , traceback

def save_names_to_PKL( pklPath , nameStringList ):
    pklStruct = {}
    print( vars() )
    for name in nameStringList:
        pklStruct[ name ] = globals()[ name ]
        
    with open( pklPath , "wb" ) as f:
        try:
            print( "Pickling" , pklPath , end = '... ' )
            pickle.dump( pklStruct , f )
            print( "Success!" )
        except Exception as er:
            print( "FAILURE! :" , er )
            traceback.print_exc()
    print( "### Completed Writing" , pklPath , "###" )
    
save_names_to_PKL( 
    pklPath = "taughtPoses.pkl" , 
    nameStringList = [  
        'startPose' ,
        'M8_GraspPose' , 'M6_GraspPose' , 'M4_GraspPose' , 'GearShaft1_Setdown' , 'GearShaft2_Setdown' , 
        'BNC_Ma_Setdown' , 'KeySm_Setdown' , 'KeyLg_Setdown' , 'ShaftSm_Setdown' , 'ShaftLg_Setdown' , 
        'KeyLg_Grasp' , 'ShaftLg_Grasp' , 'BNC_Fe_Grasp' , 'ShaftSm_Grasp' , 
        'GearLg_Grasp' , 'GearLg_Grasp' , 'Ether_setdown' , 
        'M8_1_mag' , 'M8_2_mag' , 'M8_3_mag' , 'M8_4_mag' , 'M8_5_mag' , 'M8_6_mag' ,
        'M6_1_mag' , 'M6_2_mag' , 'M6_3_mag' , 'M6_4_mag' , 'M6_5_mag' , 'M6_6_mag' ,
        'M4_1_mag' , 'M4_2_mag' , 'M4_3_mag' , 'M4_4_mag' , 'M4_5_mag' , 'M4_6_mag' ,
        'releasePose' , 'SAFE_POSE' , 'midDrillPose' , 
        '_LRG_GEAR_DEX' , '_SML_GEAR_DEX', 'aboveTableDrill'
    ]
)